In [ ]:
import os
import requests
import zipfile
import tarfile
import shutil
import math
import json
import time
import sys
import cv2
import string
import re
import subprocess
import hashlib
import numpy as np
import pandas as pd
from glob import glob
import collections
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

# Tensorflow
import tensorflow as tf

# sklearn
from sklearn.model_selection import train_test_split

# Dask
import dask
import dask.dataframe as dd
import dask.delayed as delayed
from dask.diagnostics import ProgressBar

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# Habilitar/Deshabilitar Eager Execution
# Referencia: https://www.tensorflow.org/guide/eager
# La ejecución ansiosa de TensorFlow es un entorno de programación imperativa que evalúa operaciones de inmediato,
# sin construir gráficos

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print("tensorflow version", tf.__version__)
print("Eager Execution Enabled:", tf.executing_eagerly())

# Obtener el número de réplicas
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Mejor rendimiento con la API tf.data
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

tensorflow version 2.17.1
Eager Execution Enabled: True
Number of replicas: 1
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
All Physical Devices [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

SecretNotFoundError: Secret KAGGLE_KEY does not exist.

In [ ]:
!kaggle competitions download -c amex-default-prediction

!unzip -q amex-default-prediction.zip -d ./amex-data



100% 20.5G/20.5G [04:04<00:00, 80.3MB/s]
100% 20.5G/20.5G [04:04<00:00, 89.9MB/s]


In [ ]:
!pip install dask[complete] scikit-learn


INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
import dask.dataframe as dd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Cargar los datos con Dask
train_data_path = 'amex-data/train_data.csv'
train_labels_path = 'amex-data/train_labels.csv'


X_dask = dd.read_csv(train_data_path)
y_dask = dd.read_csv(train_labels_path)

print(X_dask.head())
print(y_dask.head())


                                         customer_ID         S_2       P_2  \
0  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-03-09  0.938469   
1  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-04-07  0.936665   
2  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-05-28  0.954180   
3  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-06-13  0.960384   
4  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-07-16  0.947248   

       D_39       B_1       B_2       R_1       S_3      D_41       B_3  ...  \
0  0.001733  0.008724  1.006838  0.009228  0.124035  0.008771  0.004709  ...   
1  0.005775  0.004923  1.000653  0.006151  0.126750  0.000798  0.002714  ...   
2  0.091505  0.021655  1.009672  0.006815  0.123977  0.007598  0.009423  ...   
3  0.002455  0.013683  1.002700  0.001373  0.117169  0.000685  0.005531  ...   
4  0.002483  0.015193  1.000727  0.007605  0.117325  0.004653  0.009312  ...   

   D_136  D_137  D_138     D_139     D_140     D_1

In [ ]:

X_dask['customer_ID'] = X_dask['customer_ID'].astype(str)
y_dask['customer_ID'] = y_dask['customer_ID'].astype(str)

df_dask = dd.merge(X_dask, y_dask, on='customer_ID', how='inner')

df_dask.head()


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827,0


In [ ]:

target_col = ['target']
id_col = ['customer_ID']


categorical_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']


all_cols = df_dask.columns
not_categorical_cols = [col for col in all_cols if col not in categorical_cols + target_col + id_col]


numeric_cols = []
for col in not_categorical_cols:
    if df_dask[col].dtype != 'object' and df_dask[col].count().compute() > 0:
        numeric_cols.append(col)


print(f"Categorical columns: {categorical_cols}")
print(f"Numeric columns: {numeric_cols}")


In [ ]:

for col in numeric_cols:
    mean_value = df_dask[col].mean().compute()
    df_dask[col] = df_dask[col].fillna(mean_value)

df_dask[numeric_cols].head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


X = df_dask[numeric_cols].compute()
y = df_dask['target'].compute()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")


In [ ]:

clf = DecisionTreeClassifier(random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


In [ ]:
import joblib

model_filename = 'decision_tree_model.joblib'
joblib.dump(clf, model_filename)

print("Modelo guardado exitosamente.")



# mongo db


In [ ]:
import pymongo
import gridfs
import joblib

# Conectar a MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['model_database']
fs = gridfs.GridFS(db)  # Usamos GridFS para almacenar archivos binarios grandes


model_id = 'decision_tree_model'

model_file = fs.get(model_id)
model_binary = model_file.read()

with open('loaded_decision_tree_model.joblib', 'wb') as f:
    f.write(model_binary)


clf_loaded = joblib.load('loaded_decision_tree_model.joblib')

print("Modelo cargado exitosamente desde MongoDB.")

In [ ]:
import pandas as pd
import dask.dataframe as dd


test_data_dask = dd.read_csv('test_data.csv')


test_data_dask['customer_ID'] = test_data_dask['customer_ID'].astype(str)

non_categorical_cols = [col for col in test_data_dask.columns if col not in categorical_cols + ['target']]

X_test_dask = test_data_dask[non_categorical_cols].compute()


y_pred = clf_loaded.predict(X_test_dask)


print("Predicciones:", y_pred[:10])

predictions_df = pd.DataFrame({'customer_ID': test_data_dask['customer_ID'].compute(), 'target': y_pred})

predictions_df.to_csv('predictions.csv', index=False)

print("Predicciones guardadas en 'predictions.csv'.")
